In [66]:
## Importing Dependencies
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [67]:
# Setup the model
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [68]:
# Calculate Sentiment
tokens = tokenizer.encode("Tasty!", return_tensors = 'pt')
result = model(tokens)
result.logits

tensor([[-1.4277, -1.2743, -0.1505,  0.9500,  1.5098]],
       grad_fn=<AddmmBackward0>)

In [69]:
int(torch.argmax(result.logits)) +1

5

In [70]:
# Collect Reviews
r = requests.get('https://www.yelp.com/biz/henrys-hunan-restaurant-san-francisco-9')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [71]:
results

[<p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">A very unassuming exterior with really impressive food and service. I passed this restaurant hundreds of times without giving it a second look and that was a horrible mistake. One of the best Chinese restaurants in Nōe, certainly. Great use of fresh ginger and garlic to make the dishes really flavorful. Some Chinese restaurants in the city feel like they use basically the same sauce for every dish but here, but there was distinct flavors to everything we had.<br/><br/>Feels like the kind of place that old San Franciscans come all the way from North Beach for.<br/><br/>Definitely plan on doing takeout here more routinely, even though they were very nice about the fact that our kid littered their floor with rice. Sorry!</span></p>,
 <p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">First time here, and i was so happy that this restaurant was so nice and clean. When y

In [72]:
results[0].text

'A very unassuming exterior with really impressive food and service. I passed this restaurant hundreds of times without giving it a second look and that was a horrible mistake. One of the best Chinese restaurants in Nōe, certainly. Great use of fresh ginger and garlic to make the dishes really flavorful. Some Chinese restaurants in the city feel like they use basically the same sauce for every dish but here, but there was distinct flavors to everything we had.Feels like the kind of place that old San Franciscans come all the way from North Beach for.Definitely plan on doing takeout here more routinely, even though they were very nice about the fact that our kid littered their floor with rice. Sorry!'

In [73]:
# Load Review into DataFrame and Score
import pandas as pd
import numpy as np
df = pd.DataFrame(reviews, columns = ['review'])

In [74]:
df['review'].iloc[0]

'A very unassuming exterior with really impressive food and service. I passed this restaurant hundreds of times without giving it a second look and that was a horrible mistake. One of the best Chinese restaurants in Nōe, certainly. Great use of fresh ginger and garlic to make the dishes really flavorful. Some Chinese restaurants in the city feel like they use basically the same sauce for every dish but here, but there was distinct flavors to everything we had.Feels like the kind of place that old San Franciscans come all the way from North Beach for.Definitely plan on doing takeout here more routinely, even though they were very nice about the fact that our kid littered their floor with rice. Sorry!'

In [75]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [76]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,A very unassuming exterior with really impress...,4
1,"First time here, and i was so happy that this ...",5
2,Try Diana's meat pie and you'll go back over a...,4
3,It's really 3.5 stars. The service is slow an...,3
4,Great Chinese food. Tastes a lot more fresh an...,4
5,I forgot how much I like Henry's Hunan Restaur...,3
6,We came here for dinner. There were perhaps 2 ...,2
7,Henry's is a San Francisco tradition and this ...,5
8,This was my second time trying Henry's and it'...,5
9,"Great hunan food, nice place parking is a pain...",5


In [79]:
# Give Overall Sentiment
df['sentiment'].value_counts(normalize = True) * 100


sentiment
5    40.0
4    30.0
3    20.0
2    10.0
Name: proportion, dtype: float64